In [1]:
import numpy
print(numpy.__version__)

1.24.3


In [2]:
import time
import pandas as pd
import numpy as np
import scipy.sparse as sps
import random as rnd

from scipy.sparse import *

In [8]:
urm_path = './content/data_train.csv'
urm_all_df = pd.read_csv(filepath_or_buffer=urm_path,
                                sep=",",
                                header=0,
                                dtype={0:int, 1:int, 2:float},
                                engine='python')

urm_all_df.columns = ["UserID", "ItemID", "Interaction"]
print ("The number of interactions is {}".format(len(urm_all_df)))


The number of interactions is 478730


In [9]:
userID_unique = urm_all_df["UserID"].unique()
itemID_unique = urm_all_df["ItemID"].unique()

n_users = len(userID_unique)
n_items = len(itemID_unique)
n_interactions = len(urm_all_df)

In [10]:
urm_all = sps.coo_matrix((urm_all_df["Interaction"].values,
                          (urm_all_df["UserID"].values, urm_all_df["ItemID"].values)))

urm_all

<13025x22348 sparse matrix of type '<class 'numpy.float64'>'
	with 478730 stored elements in COOrdinate format>

In [11]:
URM_all = urm_all

### Step 1: Split the data and create the evaluator objects

In [4]:
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

In [12]:
from Evaluation.Evaluator import EvaluatorHoldout

URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.8)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 2974 (22.8%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 2625 (20.2%) Users that have less than 1 test interactions


### Step 2: Define hyperparameter set for the desired model, in this case rp3beta

In [13]:
from skopt.space import Real, Integer, Categorical

hyperparameters_range_dictionary = {
    "topK": Integer(0, 500),
    "l2_norm": Real(0.00001, 0.9999)
}

### Step 3: Create SearchBayesianSkopt object, providing the desired recommender class and evaluator objects

In [14]:
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender
from HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt

recommender_class = EASE_R_Recommender

hyperparameterSearch = SearchBayesianSkopt(recommender_class,
                                         evaluator_validation=evaluator_validation,
                                         evaluator_test=evaluator_test)

Tensorflow is not available


### Step 4: Provide the data needed to create an instance of the model, one trained only on URM_train, the other on URM_train_validation

In [15]:
from HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
  
recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train],     # For a CBF model simply put [URM_train, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {},
    EARLYSTOPPING_KEYWORD_ARGS = {},
)

In [16]:
recommender_input_args_last_test = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train_validation],     # For a CBF model simply put [URM_train_validation, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {},
    EARLYSTOPPING_KEYWORD_ARGS = {},
)

### Step 5: Create a result folder and select the number of cases (50 with 30% random is a good number)

In [18]:
import os

output_folder_path = "result_experiments/easer"

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)
    
n_cases = 10  # using 10 as an example
n_random_starts = int(n_cases*0.3)
metric_to_optimize = "MAP"   
cutoff_to_optimize = 10

### Step 5: Run!

In [19]:
hyperparameterSearch.search(recommender_input_args,
                       recommender_input_args_last_test = recommender_input_args_last_test,
                       hyperparameter_search_space = hyperparameters_range_dictionary,
                       n_cases = n_cases,
                       n_random_starts = n_random_starts,
                       save_model = "last",
                       output_folder_path = output_folder_path, # Where to save the results
                       output_file_name_root = recommender_class.RECOMMENDER_NAME, # How to call the files
                       metric_to_optimize = metric_to_optimize,
                       cutoff_to_optimize = cutoff_to_optimize,
                      )

Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'topK': 172, 'l2_norm': 0.5920600794727431}
EASE_R_Recommender: URM Detected 836 ( 6.4%) users with no interactions.
EASE_R_Recommender: URM Detected 441 ( 2.0%) items with no interactions.
EASE_R_Recommender: Fitting model... 
SearchBayesianSkopt: Search for 'EASE_R_Recommender' interrupted due to MemoryError.
SearchBayesianSkopt: Search interrupted due to ValueError. The evaluated configurations may have had all the same value.



Traceback (most recent call last):
  File "d:\Programmazione\RecSys\Challenge\RecSys_Course_AT_PoliMi\HyperparameterTuning\SearchBayesianSkopt.py", line 340, in search
    self.result = gp_minimize(self._objective_function_list_input,
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Anaconda\Lib\site-packages\skopt\optimizer\gp.py", line 259, in gp_minimize
    return base_minimize(
           ^^^^^^^^^^^^^^
  File "d:\Anaconda\Lib\site-packages\skopt\optimizer\base.py", line 300, in base_minimize
    result = optimizer.tell(next_x, next_y)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Anaconda\Lib\site-packages\skopt\optimizer\optimizer.py", line 483, in tell
    self._check_y_is_valid(x, y)
  File "d:\Anaconda\Lib\site-packages\skopt\optimizer\optimizer.py", line 636, in _check_y_is_valid
    raise ValueError("`func` should return a scalar")
ValueError: `func` should return a scalar


### Check the best model

In [ ]:
from Recommenders.DataIO import DataIO

data_loader = DataIO(folder_path = output_folder_path)
search_metadata = data_loader.load_data(recommender_class.RECOMMENDER_NAME + "_metadata.zip")

search_metadata.keys()

dict_keys(['algorithm_name_recommender', 'algorithm_name_search', 'cutoff_to_optimize', 'exception_list', 'hyperparameters_best', 'hyperparameters_best_index', 'hyperparameters_df', 'metric_to_optimize', 'result_on_earlystopping_df', 'result_on_last', 'result_on_test_best', 'result_on_test_df', 'result_on_validation_best', 'result_on_validation_df', 'time_df', 'time_on_last_df', 'time_on_test_avg', 'time_on_test_total', 'time_on_train_avg', 'time_on_train_total', 'time_on_validation_avg', 'time_on_validation_total'])

In [ ]:
hyperparameters_df = search_metadata["hyperparameters_df"]
hyperparameters_df

,topK,alpha,beta,min_rating
0,191,0.153205,0.415763,0.096262
1,195,0.180048,0.430709,0.051321
2,191,0.244912,0.543993,0.076863
3,201,0.283493,0.599987,0.13137
4,206,0.101982,0.490537,0.147859
5,210,0.287552,0.4,0.074654
6,191,0.261103,0.465796,0.072548
7,197,0.117521,0.599472,0.051687
8,190,0.3,0.423306,0.15
9,196,0.251972,0.428469,0.084991


In [ ]:
result_on_validation_df = search_metadata["result_on_validation_df"]
result_on_validation_df

,,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
,cutoff,,,,,,,,,,,,,,,,,,,,,
0,10,0.058136,0.116532,0.10449,0.025138,0.051334,0.173668,0.097432,0.074707,0.396335,0.208187,...,0.770979,0.305566,0.770979,0.111439,10.100322,0.994639,0.325472,0.778796,2.428833,0.37993
1,10,0.058116,0.116293,0.104162,0.025094,0.051048,0.172938,0.097078,0.074606,0.396236,0.207534,...,0.770979,0.305489,0.770979,0.118544,10.186441,0.994805,0.346222,0.785437,2.38934,0.383111
2,10,0.050856,0.100886,0.090243,0.021593,0.043064,0.149166,0.083338,0.065053,0.351524,0.178725,...,0.770979,0.271017,0.770979,0.178934,11.06193,0.996066,0.522599,0.852942,1.930246,0.424393
3,10,0.041605,0.083005,0.074299,0.017612,0.035111,0.12312,0.068345,0.053341,0.294065,0.146602,...,0.770979,0.226718,0.770979,0.218929,11.959534,0.998703,0.63941,0.922153,1.166918,0.469281
4,10,0.053595,0.106647,0.095316,0.02306,0.046545,0.160891,0.089165,0.068611,0.368353,0.191829,...,0.770979,0.283992,0.770979,0.147085,10.550136,0.995266,0.429581,0.81348,2.205501,0.401919
5,10,0.058594,0.118221,0.106088,0.025169,0.051889,0.173922,0.098321,0.075492,0.399622,0.208489,...,0.770979,0.3081,0.770979,0.112266,10.04182,0.994049,0.327885,0.774286,2.509984,0.377418
6,10,0.057409,0.114672,0.102646,0.024543,0.049791,0.169197,0.09523,0.073635,0.391058,0.203013,...,0.770979,0.301497,0.770979,0.135242,10.395307,0.995202,0.39499,0.801542,2.289242,0.391372
7,10,0.03801,0.077083,0.06951,0.015759,0.032249,0.111741,0.062754,0.049146,0.272754,0.132144,...,0.770979,0.210288,0.770979,0.201885,11.925596,0.998993,0.58963,0.919536,1.034704,0.478989
8,10,0.058813,0.11776,0.105507,0.025283,0.051683,0.174359,0.098208,0.075526,0.398825,0.209161,...,0.770979,0.307486,0.770979,0.118274,10.16778,0.994621,0.345433,0.783998,2.419057,0.381554


In [ ]:
result_best_on_test = search_metadata["result_on_last"]
result_best_on_test

,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.082896,0.147385,0.125281,0.041163,0.072364,0.240702,0.132361,0.099774,0.482982,0.311432,...,0.805298,0.388944,0.805298,0.083438,9.840057,0.994482,0.24089,0.7581,2.450768,0.295912


In [ ]:
best_hyperparameters = search_metadata["hyperparameters_best"]
best_hyperparameters

{'topK': 190, 'alpha': 0.3, 'beta': 0.4233057604632479, 'min_rating': 0.15}

In [ ]:
time_df = search_metadata["time_df"]
time_df

,train,validation,test
0,19.994527,10.693831,11.469868
1,19.318479,10.554817,NaN
2,19.42883,10.508951,NaN
3,20.135541,10.662126,NaN
4,20.311627,10.610811,NaN
5,20.657636,10.510118,11.140869
6,19.763679,10.71585,NaN
7,19.710493,10.557808,NaN
8,19.525661,10.504816,11.104589
9,19.697504,10.715818,NaN


In [ ]:
exception_list = search_metadata["exception_list"]
exception_list

[None, None, None, None, None, None, None, None, None, None]